This box will set up the name of the slice and set the expected resources<br>

In [2]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

exp_requires = {'core': 16*10, 'nic': 20,'disk':20*10 }
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_requires['core']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_requires['nic']) and (fablib.resources.get_disk_available(site_name) > 1.2*exp_requires['disk']) ):
        break


import os
slice_name="IP_Test_" + os.getenv('NB_USER')

try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
except:
    print("You don't have a slice named %s yet." % slice_name)
    slice = fablib.new_slice(name=slice_name)

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
Bastion Username,maross6_0000108606
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


You already have a slice by this name!


<br><br>Next we name and set the networks and interfaces<br>

In [2]:
# this cell sets up the hosts and router
node_names = ["c1","c2","c3","p1","p2","p3", "r1", "r2", "r3","r4"]
for n in node_names:
    slice.add_node(name=n, site=site_name, cores=16, ram=32, disk=20, image='default_ubuntu_22')
    
# this cell sets up the network links
nets = [
    {"name": "net1",   "nodes": ["r1","c1","c2"]},
    {"name": "net2",   "nodes": ["r2","p1"]},
    {"name": "net3",   "nodes": ["r3","p2"]},
    {"name": "net4",   "nodes": ["r4","p3","c3"]},
    {"name": "net12",   "nodes": ["r1","r2"]},
    {"name": "net23",   "nodes": ["r2","r3"]},
    {"name": "net34",   "nodes": ["r3","r4"]},
    {"name": "net41",   "nodes": ["r4","r1"]},
]
for n in nets:
    ifaces = [slice.get_node(node).add_component(model="NIC_Basic", name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

<br><br>Request the slice and wait until it has been reserved<br>

In [3]:
slice.submit()


Retry: 15, Time: 1000 sec


ID,09081472-d4ab-4beb-bc11-a3e2f9434f2f
Name,IP_Test_maross6
Lease Expiration (UTC),2023-12-08 06:07:14 +0000
Lease Start (UTC),2023-12-07 06:07:14 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
ee4fdb89-ba6a-4875-a22f-2dbdad6605a6,c1,16,32,100,default_ubuntu_22,qcow2,star-w3.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fea2:d4f0,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fea2:d4f0,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
93892ff8-ad63-4743-9604-71056feb34f2,c2,16,32,100,default_ubuntu_22,qcow2,star-w3.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fe9c:ee62,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe9c:ee62,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
bb559889-0cc9-4d8c-988f-b11065103b72,c3,16,32,100,default_ubuntu_22,qcow2,star-w4.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fe0a:3333,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe0a:3333,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
baf94412-19c9-4429-a787-fff653433034,p1,16,32,100,default_ubuntu_22,qcow2,star-w4.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:feee:52bb,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:feee:52bb,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
0b112e41-b338-4c37-ab17-67dc3da0d3a5,p2,16,32,100,default_ubuntu_22,qcow2,star-w4.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fed9:cb41,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fed9:cb41,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
87a053bb-f026-46c8-b9b1-5aa313b14901,p3,16,32,100,default_ubuntu_22,qcow2,star-w4.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:febd:7f38,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:febd:7f38,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
5d93e769-5b3b-4c9f-82fa-ffda25cd4e71,r1,16,32,100,default_ubuntu_22,qcow2,star-w4.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:feb5:399a,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:feb5:399a,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
8268703f-cf01-47e0-89b3-90d416a203c5,r2,16,32,100,default_ubuntu_22,qcow2,star-w4.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fe54:e2c5,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe54:e2c5,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
5b6be7c6-d159-423b-9854-115017bda483,r3,16,32,100,default_ubuntu_22,qcow2,star-w4.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fed2:9fb7,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fed2:9fb7,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
b040063d-5537-499c-9113-a76a3ff943ff,r4,16,32,100,default_ubuntu_22,qcow2,star-w2.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fe05:8761,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe05:8761,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/w

ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
89958e0a-5a41-4bc7-9331-0d3e6d0b779a,net1,L2,L2Bridge,STAR,None,None,Active,
0096a0f5-e635-4f78-a91b-3049c82c46ac,net12,L2,L2Bridge,STAR,None,None,Active,
83775f0e-1c81-43bb-93a3-374e7d8edb3e,net2,L2,L2Bridge,STAR,None,None,Active,
c0c447c8-2566-45be-8a02-700777a88e7d,net23,L2,L2Bridge,STAR,None,None,Active,
f8c53b09-14b6-4692-a5d5-a88a1522aae2,net3,L2,L2Bridge,STAR,None,None,Active,
31fb1729-7ffa-4d98-9486-403c17bf0314,net34,L2,L2Bridge,STAR,None,None,Active,
297fe3ea-237e-4aae-9fd1-44a6c904bf57,net4,L2,L2Bridge,STAR,None,None,Active,
1d12ee32-5f2e-4a2c-8952-4976be7d5a77,net41,L2,L2Bridge,STAR,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
c1-net1-p1,p1,c1,net1,100,config,,2E:94:FE:A6:C8:24,enp7s0,enp7s0,None,4
c2-net1-p1,p1,c2,net1,100,config,,2E:58:F5:C7:14:9A,enp7s0,enp7s0,None,4
c3-net4-p1,p1,c3,net4,100,config,,02:67:7F:D4:71:C5,enp7s0,enp7s0,None,4
p1-net2-p1,p1,p1,net2,100,config,,02:8D:B7:13:2E:64,enp7s0,enp7s0,None,4
p2-net3-p1,p1,p2,net3,100,config,,02:D3:FF:39:61:4C,enp7s0,enp7s0,None,4
p3-net4-p1,p1,p3,net4,100,config,,06:4B:68:59:D7:76,enp7s0,enp7s0,None,4
r1-net41-p1,p1,r1,net41,100,config,,0A:2A:39:6C:DC:07,enp9s0,enp9s0,None,4
r1-net1-p1,p1,r1,net1,100,config,,06:FC:6B:FF:76:34,enp8s0,enp8s0,None,4
r1-net12-p1,p1,r1,net12,100,config,,06:9B:74:B6:7C:0D,enp7s0,enp7s0,None,4
r2-net2-p1,p1,r2,net2,100,config,,0A:EF:92:B2:49:F1,enp8s0,enp8s0,None,4



Time to print interfaces 1025 seconds


'09081472-d4ab-4beb-bc11-a3e2f9434f2f'

In [4]:
slice.wait_ssh(progress=True)

Waiting for slice . Slice state: StableOK
Waiting for ssh in slice . ssh successful


True

In [6]:
from ipaddress import ip_address, IPv4Address, IPv4Network
import json


if_conf = {
    "c1-net1-p1":  {"addr": "10.0.0.1", "subnet": "10.0.0.0/16"},
    "c2-net1-p1":  {"addr": "10.0.0.2", "subnet": "10.0.0.0/16"},
    "r1-net1-p1":  {"addr": "10.0.1.1", "subnet": "10.0.0.0/16"},
    
    "p1-net2-p1":  {"addr": "20.0.2.1", "subnet": "20.0.0.0/16"},
    "r2-net2-p1":  {"addr": "20.0.1.2", "subnet": "20.0.0.0/16"},
    
    "p2-net3-p1":  {"addr": "30.0.2.2", "subnet": "30.0.0.0/16"},
    "r3-net3-p1":  {"addr": "30.0.1.3", "subnet": "30.0.0.0/16"},
    
    "p3-net4-p1":  {"addr": "40.0.2.3", "subnet": "40.0.0.0/16"},
    "c3-net4-p1":  {"addr": "40.0.0.3", "subnet": "40.0.0.0/16"},
    "r4-net4-p1":  {"addr": "40.0.1.4", "subnet": "40.0.0.0/16"},
    
    "r1-net12-p1":  {"addr": "12.0.1.1", "subnet": "12.0.0.0/16"},
    "r2-net12-p1":  {"addr": "12.0.1.2", "subnet": "12.0.0.0/16"},
    
    "r2-net23-p1":  {"addr": "23.0.1.2", "subnet": "23.0.0.0/16"},
    "r3-net23-p1":  {"addr": "23.0.1.3", "subnet": "23.0.0.0/16"},
    
    "r3-net34-p1":  {"addr": "34.0.1.3", "subnet": "34.0.0.0/16"},
    "r4-net34-p1":  {"addr": "34.0.1.4", "subnet": "34.0.0.0/16"},
    
    "r4-net41-p1":  {"addr": "41.0.1.4", "subnet": "41.0.0.0/16"},
    "r1-net41-p1":  {"addr": "41.0.1.1", "subnet": "41.0.0.0/16"}
}

for iface in slice.get_interfaces():
    if_name = iface.get_name()
    iface.ip_addr_add(addr=if_conf[if_name]['addr'], subnet=IPv4Network(if_conf[if_name]['subnet']))
    
for n in ['r1','r2','r3','r4']:
    slice.get_node(name=n).execute("sudo sysctl -w net.ipv4.ip_forward=1")
    
for iface in slice.get_interfaces():
    iface.ip_link_up()
    

rt_conf = [
    {"name": "r1",  "addr": "20.0.0.0/16", "gw": "12.0.1.2"},
    {"name": "r1",  "addr": "40.0.0.0/16", "gw": "41.0.1.4"},
    {"name": "r1",  "addr": "30.0.0.0/16", "gw": "12.0.1.2"},
    
    {"name": "r2",  "addr": "10.0.0.0/16", "gw": "12.0.1.1"},
    {"name": "r2",  "addr": "30.0.0.0/16", "gw": "23.0.1.3"},
    {"name": "r2",  "addr": "40.0.0.0/16", "gw": "23.0.1.3"},
    
    {"name": "r3",  "addr": "20.0.0.0/16", "gw": "23.0.1.2"},
    {"name": "r3",  "addr": "40.0.0.0/16", "gw": "34.0.1.4"},
    {"name": "r3",  "addr": "10.0.0.0/16", "gw": "34.0.1.4"},
    
    {"name": "r4",  "addr": "10.0.0.0/16", "gw": "41.0.1.1"},
    {"name": "r4",  "addr": "30.0.0.0/16", "gw": "34.0.1.3"},
    {"name": "r4",  "addr": "20.0.0.0/16", "gw": "34.0.1.3"}
    
]

for rt in rt_conf:
    slice.get_node(name=rt['name']).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])
    
    
for n in ["c1","c2","c3","p1","p2","p3", "r1", "r2", "r3","r4"]:
    slice.get_node(name=n).execute("sudo apt update; sudo apt -y install net-tools", quiet=True)
    
    
stored_interfaces = slice.list_interfaces(output="json",fields=["name","dev"],quiet=True)
stored_interfaces_json = json.loads(stored_interfaces)

c1_dev = ''
c2_dev = ''
c3_dev = ''
p1_dev = ''
p2_dev = ''
p3_dev = ''

for entry in stored_interfaces_json:
    if(entry['name'] == 'c1-net1-p1'):
        c1_dev = entry['dev']
    elif(entry['name'] == 'c2-net1-p1'):
        c2_dev = entry['dev']
    elif(entry['name'] == 'cc3-net4-p1'):
        c3_dev = entry['dev']
    elif(entry['name'] == 'p1-net2-p1'):
        p1_dev = entry['dev']
    elif(entry['name'] == 'p2-net3-p1'):
        p2_dev = entry['dev']
    elif(entry['name'] == 'p3-net4-p1'):
        p3_dev = entry['dev']
    

slice.get_node(name="c1").execute("sudo route add default gw 10.0.1.1 "+c1_dev, quiet=False)
slice.get_node(name="c2").execute("sudo route add default gw 10.0.1.1 "+c2_dev, quiet=False)
slice.get_node(name="p1").execute("sudo route add default gw 20.0.1.2 "+p1_dev, quiet=False)
slice.get_node(name="p2").execute("sudo route add default gw 30.0.1.3 "+p2_dev, quiet=False)
slice.get_node(name="c3").execute("sudo route add default gw 40.0.1.4 "+c3_dev, quiet=False)
slice.get_node(name="p3").execute("sudo route add default gw 40.0.1.4 "+p3_dev, quiet=False)
    
import pandas as pd
pd.set_option('display.max_colwidth', None)
ssh_str = 'ssh -i ' + slice.get_slice_private_key_file() + \
    ' -J ' + fablib.get_bastion_username() + '@' + fablib.get_bastion_public_addr() + \
    ' -F /home/fabric/work/fabric_config/ssh_config '
slice_info = [{'Name': n.get_name(), 'SSH command': ssh_str + n.get_username() + '@' + str(n.get_management_ip())} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

net.ipv4.ip_forward = 1
net.ipv4.ip_forward = 1
net.ipv4.ip_forward = 1
net.ipv4.ip_forward = 1


,SSH command
Name,
c1,ssh -i /home/fabric/work/fabric_config/slice_key -J maross6_0000108606@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fea2:d4f0
c2,ssh -i /home/fabric/work/fabric_config/slice_key -J maross6_0000108606@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe9c:ee62
c3,ssh -i /home/fabric/work/fabric_config/slice_key -J maross6_0000108606@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe0a:3333
p1,ssh -i /home/fabric/work/fabric_config/slice_key -J maross6_0000108606@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:feee:52bb
p2,ssh -i /home/fabric/work/fabric_config/slice_key -J maross6_0000108606@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fed9:cb41
p3,ssh -i /home/fabric/work/fabric_config/slice_key -J maross6_0000108606@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:febd:7f38
r1,ssh -i /home/fabric/work/fabric_config/slice_key -J maross6_0000108606@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:feb5:399a
r2,ssh -i /home/fabric/work/fabric_config/slice_key -J maross6_0000108606@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe54:e2c5
r3,ssh -i /home/fabric/work/fabric_config/slice_key -J maross6_0000108606@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fed2:9fb7


Create a visual representation of the network

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)

nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

Test a few connections through the network

In [7]:
slice.get_node(name="c1").execute("ping 30.0.2.2 -c 5", quiet=False)

PING 30.0.2.2 (30.0.2.2) 56(84) bytes of data.
64 bytes from 30.0.2.2: icmp_seq=1 ttl=61 time=0.612 ms
64 bytes from 30.0.2.2: icmp_seq=2 ttl=61 time=0.257 ms
64 bytes from 30.0.2.2: icmp_seq=3 ttl=61 time=0.263 ms
64 bytes from 30.0.2.2: icmp_seq=4 ttl=61 time=0.270 ms
64 bytes from 30.0.2.2: icmp_seq=5 ttl=61 time=0.343 ms

--- 30.0.2.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4077ms
rtt min/avg/max/mdev = 0.257/0.349/0.612/0.135 ms


('PING 30.0.2.2 (30.0.2.2) 56(84) bytes of data.\n64 bytes from 30.0.2.2: icmp_seq=1 ttl=61 time=0.612 ms\n64 bytes from 30.0.2.2: icmp_seq=2 ttl=61 time=0.257 ms\n64 bytes from 30.0.2.2: icmp_seq=3 ttl=61 time=0.263 ms\n64 bytes from 30.0.2.2: icmp_seq=4 ttl=61 time=0.270 ms\n64 bytes from 30.0.2.2: icmp_seq=5 ttl=61 time=0.343 ms\n\n--- 30.0.2.2 ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 4077ms\nrtt min/avg/max/mdev = 0.257/0.349/0.612/0.135 ms\n',
 '')

In [8]:
slice.get_node(name="p1").execute("ping 40.0.2.3 -c 5", quiet=False)

PING 40.0.2.3 (40.0.2.3) 56(84) bytes of data.
64 bytes from 40.0.2.3: icmp_seq=1 ttl=61 time=0.469 ms
64 bytes from 40.0.2.3: icmp_seq=2 ttl=61 time=0.264 ms
64 bytes from 40.0.2.3: icmp_seq=3 ttl=61 time=0.285 ms
64 bytes from 40.0.2.3: icmp_seq=4 ttl=61 time=0.258 ms
64 bytes from 40.0.2.3: icmp_seq=5 ttl=61 time=0.254 ms

--- 40.0.2.3 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4100ms
rtt min/avg/max/mdev = 0.254/0.306/0.469/0.082 ms


('PING 40.0.2.3 (40.0.2.3) 56(84) bytes of data.\n64 bytes from 40.0.2.3: icmp_seq=1 ttl=61 time=0.469 ms\n64 bytes from 40.0.2.3: icmp_seq=2 ttl=61 time=0.264 ms\n64 bytes from 40.0.2.3: icmp_seq=3 ttl=61 time=0.285 ms\n64 bytes from 40.0.2.3: icmp_seq=4 ttl=61 time=0.258 ms\n64 bytes from 40.0.2.3: icmp_seq=5 ttl=61 time=0.254 ms\n\n--- 40.0.2.3 ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 4100ms\nrtt min/avg/max/mdev = 0.254/0.306/0.469/0.082 ms\n',
 '')

Install Apache on producers and set data

In [9]:
for n in ['p1','p2','p3']:
    slice.get_node(name=n).execute("sudo apt-get -y -q install apache2")
    slice.get_node(name=n).execute("sudo service apache2 start")
    slice.get_node(name=n).execute("sudo rm /var/www/html/index.html")
    slice.get_node(name=n).execute("mkdir RunData")
    slice.get_node(name=n).execute("head -c 100M </dev/urandom > RunData/100Mtest")
    slice.get_node(name=n).execute("head -c 500K </dev/urandom > RunData/500Ktest")
    slice.get_node(name=n).execute("head -c 1G </dev/urandom > RunData/1Gtest")
    slice.get_node(name=n).execute("sudo cp -r RunData /var/www/html/Data")

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  apache2-bin apache2-data apache2-utils bzip2 libapr1 libaprutil1
  libaprutil1-dbd-sqlite3 libaprutil1-ldap liblua5.3-0 mailcap mime-support
  ssl-cert
Suggested packages:
  apache2-doc apache2-suexec-pristine | apache2-suexec-custom www-browser
  bzip2-doc
The following NEW packages will be installed:
  apache2 apache2-bin apache2-data apache2-utils bzip2 libapr1 libaprutil1
  libaprutil1-dbd-sqlite3 libaprutil1-ldap liblua5.3-0 mailcap mime-support
  ssl-cert
0 upgraded, 13 newly installed, 0 to remove and 117 not upgraded.
Need to get 2139 kB of archives.
After this operation, 8518 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates/main amd64 libapr1 amd64 1.7.0-8ubuntu0.22.04.1 [108 kB]
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates/main amd64 libaprutil1 amd64 1.6.1-5ubuntu

Run test to download data and then also clean up the created files to allow multiple runs(ran 5 times for the paper)

In [12]:
import json

#Run C1 500KB test
slice.get_node(name="c1").execute("echo C1-P2 500KB run:", quiet=False)
slice.get_node(name="c1").execute("time wget -q 30.0.2.2/Data/500Ktest", quiet=False)

#Run C2 500KB test
slice.get_node(name="c2").execute("echo C2-P2 500KB run:", quiet=False)
slice.get_node(name="c2").execute("time wget -q 30.0.2.2/Data/500Ktest", quiet=False)

#Run C1 100MB test
slice.get_node(name="c1").execute("echo C1-P2 100MB run:", quiet=False)
slice.get_node(name="c1").execute("time wget -q 30.0.2.2/Data/100Mtest", quiet=False)

#Run C2 100MB test
slice.get_node(name="c2").execute("echo C2-P2 100MB run:", quiet=False)
slice.get_node(name="c2").execute("time wget -q 30.0.2.2/Data/100Mtest", quiet=False)

#Run C1 1GB test
slice.get_node(name="c1").execute("echo C1-P2 1GB run:", quiet=False)
slice.get_node(name="c1").execute("time wget -q 30.0.2.2/Data/1Gtest", quiet=False)

#Run C2 1GB test
slice.get_node(name="c2").execute("echo C2-P2 1GB run:", quiet=False)
slice.get_node(name="c2").execute("time wget -q 30.0.2.2/Data/1Gtest", quiet=False)

#Run C3 500KB test
slice.get_node(name="c3").execute("echo C3-P1 500KB run:", quiet=False)
slice.get_node(name="c3").execute("time wget -q 20.0.2.1/Data/500Ktest", quiet=False)

#Run C3 100MB test
slice.get_node(name="c3").execute("echo C3-P1 100MB run:", quiet=False)
slice.get_node(name="c3").execute("time wget -q 20.0.2.1/Data/100Mtest", quiet=False)

#Run C3 1GB test
slice.get_node(name="c3").execute("echo C3-P1 1GB run:", quiet=False)
slice.get_node(name="c3").execute("time wget -q 20.0.2.1/Data/1Gtest", quiet=False)

#Run C2 Down Node test
slice.get_node(name="c2").execute("echo C2-P3 with R1-R4 Down run:", quiet=False)

int_dev = ''
stored_interfaces = slice.list_interfaces(output="json",fields=["name","dev"],quiet=True)
stored_interfaces_json = json.loads(stored_interfaces)
for entry in stored_interfaces_json:
    if(entry['name'] == 'r4-net41-p1'):
        int_dev = entry['dev']
        
slice.get_node(name="r4").execute("sudo ifconfig " + int_dev + " down", quiet=False)

slice.get_node(name="c2").execute("time wget 40.0.2.3/Data/500Ktest", quiet=False)

slice.get_node(name="r4").execute("sudo ifconfig " + int_dev + " up", quiet=False)
slice.get_node(name="r4").execute("sudo route add -net 10.0.0.0 netmask 255.0.0.0 gw 41.0.1.1", quiet=False)


#Run C2 Non-existant test
slice.get_node(name="c2").execute("echo C2-P2 Missing Data run:", quiet=False)
slice.get_node(name="c2").execute("time wget 30.0.2.2/Data/9999Gtest", quiet=False)

#Clear all of the created files
slice.get_node(name="c1").execute("rm 500Ktest", quiet=False)
slice.get_node(name="c2").execute("rm 500Ktest", quiet=False)
slice.get_node(name="c1").execute("rm 100Mtest", quiet=False)
slice.get_node(name="c2").execute("rm 100Mtest", quiet=False)
slice.get_node(name="c1").execute("rm 1Gtest", quiet=False)
slice.get_node(name="c2").execute("rm 1Gtest", quiet=False)
slice.get_node(name="c3").execute("rm 500Ktest", quiet=False)
slice.get_node(name="c3").execute("rm 100Mtest", quiet=False)
slice.get_node(name="c3").execute("rm 1Gtest", quiet=False)

#Check to make sure the directories are clear
slice.get_node(name="c2").execute("echo Leftover Files:", quiet=False)
slice.get_node(name="c1").execute("ls", quiet=False)
slice.get_node(name="c2").execute("ls", quiet=False)
slice.get_node(name="c3").execute("ls", quiet=False)

C1-P2 500KB run:
 
real	0m0.005s
user	0m0.000s
sys	0m0.003s
 C2-P2 500KB run:
 
real	0m0.005s
user	0m0.000s
sys	0m0.003s
 C1-P2 100MB run:
 
real	0m0.156s
user	0m0.016s
sys	0m0.139s
 C2-P2 100MB run:
 
real	0m0.145s
user	0m0.004s
sys	0m0.139s
 C1-P2 1GB run:
 
real	0m1.528s
user	0m0.048s
sys	0m1.479s
 C2-P2 1GB run:
 
real	0m1.777s
user	0m0.048s
sys	0m1.724s
 C3-P1 500KB run:
 
real	0m0.005s
user	0m0.000s
sys	0m0.003s
 C3-P1 100MB run:
 
real	0m0.158s
user	0m0.004s
sys	0m0.150s
 C3-P1 1GB run:
 
real	0m1.503s
user	0m0.104s
sys	0m1.393s
 C2-P3 with R1-R4 Down run:
 --2023-12-07 07:52:02--  http://40.0.2.3/Data/500Ktest
Connecting to 40.0.2.3:80... failed: No route to host.

real	0m18.536s
user	0m0.000s
sys	0m0.003s
 C2-P2 Missing Data run:
 --2023-12-07 07:52:27--  http://30.0.2.2/Data/9999Gtest
Connecting to 30.0.2.2:80... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-12-07 07:52:27 ERROR 404: Not Found.


real	0m0.004s
user	0m0.003s
sys	0m0.000s
 Leftover Files

('', '')

Use this to extend the slice when needed

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 14 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

slice.update()
_ = slice.show()

Use this section to delete the slice when needed

In [ ]:
slice.delete()


In [ ]:
# slice should end up in "Dead" state
# re-run this cell until you see it in "Dead" state
slice.update()
_ = slice.show()